In [ ]:
!pip install ffmpeg-python
!pip install -q transformers
import torch
import librosa
import numpy as np
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!unzip -q "/content/gdrive/MyDrive/Audio.zip"

In [ ]:
audio_folder="/content/Compressed_audio/Compressed_audio/IEMOCAP_Compressed_br128_fs20/Session1/opus/"

In [ ]:
import os
data1folder=os.listdir(audio_folder)

In [ ]:
import ffmpeg
if not os.path.exists('wavs'):
    os.makedirs('wavs')
if not os.path.exists('embeddings'):
    os.makedirs('embeddings')
for i in range(0,len(data1folder)):
  print(i,data1folder[i])
  filename = data1folder[i].split('.opus')[0]
  wav_filename = './wavs/' + filename + '.wav'
  (ffmpeg.input(audio_folder+data1folder[i]).output(wav_filename).run())


  

In [ ]:
import datetime

pathAudio="./wavs"
files = librosa.util.find_files(pathAudio, ext=['wav']) 
files = np.asarray(files)

tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to("cuda")

for i,y in enumerate(files): 
  audio, rate  = librosa.load(y, sr = 16000)
  # print(str(datetime.datetime.now()).split(' ')[1])
  input_values = tokenizer(audio, return_tensors = "pt").input_values
  logits = model(input_values.to("cuda")).logits

  print("Processed,",i)
  embedding = torch.tensor(logits,requires_grad=True)
  embedding = embedding.detach().cpu().numpy()

  filename = y.split('.wav')[0].split('/')[-1]
  embedding_filename = './embeddings/' + filename + '.npy'
  np.save(embedding_filename, embedding)
  print(str(datetime.datetime.now()).split(' ')[1])



In [ ]:
!zip -r  embeddings.zip embeddings

In [ ]:
!cp embeddings.zip "/content/gdrive/MyDrive/"